In [1]:
import datetime

## SQLAlchemy

This examples are adapted from SQLAlchemy's documentation

In [2]:
import sqlite3

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import scoped_session, sessionmaker

Base = declarative_base()
DBSession = scoped_session(sessionmaker())

SIZE = 1_000_000

class Customer(Base):
    __tablename__ = "customer"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))


def init_sqlalchemy(dbname='sqlite:///sqlalchemy.db'):
    engine = create_engine(dbname, echo=False)
    DBSession.remove()
    DBSession.configure(
        bind=engine,
        autoflush=False,
        expire_on_commit=False,
    )
    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)
    return engine

def init_sqlite3(dbname):
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS customer")
    c.execute(
        "CREATE TABLE customer (id INTEGER NOT NULL, "
        "name VARCHAR(255), PRIMARY KEY(id))"
    )
    conn.commit()
    return conn

In [3]:
_ = init_sqlalchemy()
start_time = datetime.datetime.now()

for i in range(SIZE):
    customer = Customer()
    customer.name = 'NAME ' + str(i)
    customer.id = i + 1
    DBSession.add(customer)
    if i % 1000 == 0:
        DBSession.flush()
DBSession.commit()

delta_time = datetime.datetime.now() - start_time
print(f'Done SQLAlchemy ORM, in {delta_time.total_seconds():.2f} sec')

Done SQLAlchemy ORM, in 42.58 sec


In [4]:
_ = init_sqlalchemy()
start_time = datetime.datetime.now()
for chunk in range(0, SIZE, 10_000):
    DBSession.bulk_insert_mappings(
        Customer,
        [
            dict(name="NAME " + str(i), id=i + 1)
            for i in range(chunk, min(chunk + 10000, SIZE))
        ],
    )
DBSession.commit()

delta_time = datetime.datetime.now() - start_time
print(
    'Done SQLAlchemy bulk insert,'
    f' in {delta_time.total_seconds():.2f} sec'
)

Done SQLAlchemy bulk insert, in 6.77 sec


In [5]:
conn = init_sqlite3('sqlite3.db')
c = conn.cursor()
start_time = datetime.datetime.now()
for i in range(SIZE):
    row = ('NAME ' + str(i),)
    c.execute("INSERT INTO customer (name) VALUES (?)", row)
conn.commit()

delta_time = datetime.datetime.now() - start_time
print(f'Done plain sqlite, in {delta_time.total_seconds():.2f} sec')

Done plain sqlite, in 1.76 sec


In [6]:
# This part doesn't work with sqlite, you'll need something with better support for transactions
#
# from unsync import unsync
#
# def test_sqlalchemy_orm_bulk_insert_parallel():
#     _ = init_sqlalchemy()

#     @unsync
#     def persist(objects):
#         DBSession.bulk_insert_mappings(Customer, objects)
#         DBSession.commit()

#     start_time = datetime.datetime.now()
#     tasks = []
#     for chunk in range(0, SIZE, 10_000):
#         tasks.append(
#             persist(
#                 [
#                     dict(name="NAME " + str(i), id=i + 1)
#                     for i in range(chunk, min(chunk + 10000, SIZE))
#                 ]
#             )
#         )
#     # wait for persist to finish
#     [t.result() for t in tasks]

#     delta_time = datetime.datetime.now() - start_time
#     print(
#         'Done SQLAlchemy bulk insert in parallel,'
#         f' in {delta_time.total_seconds():.2f} sec'
#     )

## Numba

In [7]:
import math
import numpy as np
from numba import jit


def distance(src, dst):
    result = np.zeros_like(src)
    for i in range(len(src)):
        result[i] = math.sqrt(
            (dst[i][0] - src[i][0]) ** 2 + (dst[i][1] - src[i][1]) ** 2
        )
    return result

@jit(nopython=True)
def jit_distance(src, dst):
    result = np.zeros_like(src)
    for i in range(len(src)):
        result[i] = math.sqrt(
            (dst[i][0] - src[i][0]) ** 2 + (dst[i][1] - src[i][1]) ** 2
        )
    return result

In [8]:
start_time = datetime.datetime.now()

src = np.random.rand(1_000_000, 2)
dst = np.random.rand(1_000_000, 2)

process_data = distance(src, dst)

delta_time = datetime.datetime.now() - start_time
print(
    'Done without numba'
    f' in {delta_time.total_seconds():.2f} sec'
)

Done without numba in 2.57 sec


In [9]:
start_time = datetime.datetime.now()

src = np.random.rand(1_000_000, 2)
dst = np.random.rand(1_000_000, 2)

process_data = jit_distance(src, dst)

delta_time = datetime.datetime.now() - start_time
print(
    'Done with numba '
    f' in {delta_time.total_seconds():.2f} sec'
)

Done with numba  in 0.40 sec
